In [1]:
import sys
!{sys.executable} -m pip install pybit
!{sys.executable} -m pip install boto3
import subprocess
from pybit.unified_trading import HTTP
import boto3
import os
from pathlib import Path

from torch import optim
from torch import nn
import torch
from torch.utils.data import DataLoader, TensorDataset
!{sys.executable} -m pip install EMD-signal
from PyEMD import CEEMDAN
from builtins import RuntimeError

from numpy import dtype
from torch.nn.utils.rnn import pad_sequence

class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_length, output_size=1, *, components=3, mean=0, scale=1):
        super(GRUNetwork, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.output_length = output_length
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.components = components

        self.mean = mean
        self.scale = scale
        # x.shape = (b_size, input_size)
        # Настройка GRU слоя
        self.gru = nn.GRU(components + input_size, hidden_size, num_layers, batch_first=True, dropout=0.1)
        self.decoder = nn.GRU(output_size, hidden_size, num_layers, batch_first=True, dropout=0.1)
        # Полносвязный слой для превратить выходные данные GRU в требуемый формат
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size, series_number, series_length = x.size()
        # CEEMDAN decomposition
        ceemdan = CEEMDAN(parallel=True)

        # Для каждого временного ряда в батче проводим EMD и разбиваем его на три дополнительных временных ряда
        decomposed_series = []
        for i in range(batch_size):
            k = (x[i].cpu().numpy().squeeze() - self.mean)/self.scale
            imfs = ceemdan(k)  # проводим EMD на временном ряде
            imfs = imfs[:self.components]  # берем только три первых IMFs
            imfs = torch.cat((x[i].transpose(0, 1).cuda(), torch.tensor(imfs, dtype=torch.float32).cuda()), dim=0)
            # Добавляем -1 размер для консистенции
            decomposed_series.append(imfs.cuda())

        x_decomposed = pad_sequence(decomposed_series, batch_first=True)
        x_decomposed = x_decomposed.permute(0, 2, 1).reshape(x_decomposed.shape[0], -1, series_length+self.components).cuda()

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).cuda()
        out, h_n = self.gru(x_decomposed, h0)
        out_last = out[:, -1, :]

        outputs = []
        for _ in range(self.output_length):
            output = self.fc(out_last)
            outputs.append(output.unsqueeze(1))
            out_last = self.decoder(output.unsqueeze(1), h_n)[0].squeeze(1)

        return torch.cat(outputs, dim=1)*self.scale + self.mean

In [4]:
def main(SYMBOL):
    session = boto3.session.Session()
    s3 = session.client(
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net',
        aws_access_key_id='YCAJEEvXFkR_Vlz_q0TNbK1f7',
        aws_secret_access_key='YCPw07kPNEgOJVF0N93yReOErAl7RP0-5woP_Bgl',
        region_name='ru-central1'
    )
    
    bucket_name = 'test-actions'

    x_train_file = f'train/trainX_{SYMBOL}.pt'
    y_train_file = f'train/trainY_{SYMBOL}.pt'
    x_test_file = f'test/testX_{SYMBOL}.pt'
    y_test_file = f'test/testY_{SYMBOL}.pt'
    s3.download_file(bucket_name, x_train_file, x_train_file[6:])
    s3.download_file(bucket_name, y_train_file, y_train_file[6:])
    s3.download_file(bucket_name, x_test_file, x_test_file[5:])
    s3.download_file(bucket_name, y_test_file, y_test_file[5:])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(x_train_file[6:])
    x_train, y_train = torch.load(f'trainX_{SYMBOL}.pt', weights_only=True).to(torch.float32).cuda(), torch.load(
        f'trainY_{SYMBOL}.pt', weights_only=True).to(torch.float32).cuda()
    x_test, y_test = torch.load(f'testX_{SYMBOL}.pt', weights_only=True).to(torch.float32).cuda(), torch.load(
        f'testY_{SYMBOL}.pt', weights_only=True).to(torch.float32).cuda()

    mean, std = x_train.squeeze().reshape(-1).detach().mean().item(), x_train.squeeze().reshape(
        -1).detach().std().item()
    dataset = TensorDataset(x_train, y_train)
    validation = TensorDataset(x_test, y_test)

    batch_size = 5
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(validation, batch_size=2, shuffle=True)

    gru = GRUNetwork(1, 30, 2, 10, components=1, mean=mean, scale=std)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    gru.cuda()

    criterion = nn.MSELoss()
    optimizer = optim.Adam(gru.parameters(), lr=0.9)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Обучение модели
    num_epochs = 50
    patience = 3
    target_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        for i, (batch_x, batch_y) in enumerate(dataloader):
            gru.train()
            outputs = gru(batch_x)
            optimizer.zero_grad()
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
        scheduler.step()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
        gru.eval()  # Переключаем модель в режим оценки
        val_loss = 0.0
        with torch.no_grad():  # Выключаем вычисление градиентов
            for batch_x, batch_y in val_dataloader:
                outputs = gru(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()

        val_loss /= len(val_dataloader)  # Средняя ошибка на валидации
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {val_loss:.4f}')

        # Реализация ранней остановки
        if val_loss < target_loss:
            target_loss = val_loss
            patience_counter = 0  # Сбрасываем счетчик
        else:
            patience_counter += 1  # Увеличиваем счетчик

        # Проверяем, если сохранять, если ошибка стабильна
        if patience_counter >= patience:
            print("Ошибка валидации не уменьшается, остановка обучения.")
            break

    model_file = f'gru_{SYMBOL}.onnx'
    torch.onnx.export(gru, x_train, model_file, export_params=True)
    s3.upload_file(f'gru_{SYMBOL}.onnx', bucket_name, 'model/'+model_file)

    weights_file = f'weights_{SYMBOL}.pt'
    torch.save(gru.state_dict(), weights_file)
    s3.upload_file(weights_file, bucket_name, 'model/'+weights_file)

In [ ]:
# Создайте объект клиента API
client = HTTP()

# Получите информацию о рынке
market_data = client.get_tickers(category='spot')

# Отсортируйте токены по объему торгов, чтобы найти самые популярные
sorted_tickers = sorted(filter(lambda x: x.get('symbol').endswith('USDT') and not x.get('symbol').startswith('USD'), market_data.get('result', {}).get('list', [])), key=lambda x: float(x['turnover24h']), reverse=True)

# Выведите топ несколько популярных токенов
top_tickers = sorted_tickers[:10]  # например, топ 5
#for ticker in top_tickers:
#    print(f"Токен: {ticker['symbol']}, Объём за 24ч: {ticker['turnover24h']}")

symbols = [ticker['symbol'] for ticker in top_tickers]

for symbol in symbols:
    main(symbol)

trainX_BTCUSDT.pt
